## Данные: 
Данные из AppMetrica компании по доставке продуктов

date – дата совершения события\
event - событие\
os_name – платформа пользователя\
city – город пользователя\
device_id – идентификатор устройства пользователя\
urm_source – канал, с которого пришел пользователь

## Задачи:
Отобразить MAU по месяцам\
Отобразить кол-во установок по месяцам\
Провести анализ CR узнать для какой когорты было максимальное значение\
С какого платного маркетингового канала пришло больше всего новых пользователей?\
CR в покупку у разных маркетинговых каналов\
Медианный чек у каналов привлечения\
Суммарная прибыль у каналов привлечения



In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [376]:
df = pd.read_csv('./KC_case_data .csv')

In [6]:
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads


In [9]:
df.date = pd.to_datetime(df.date)

In [21]:
df['month'] = df.date.dt.month

In [29]:
df.groupby('month')['device_id'].nunique()
#MAU по месяцам

month
1    99161
2    75032
3    74623
Name: device_id, dtype: int64

In [33]:
df.query("event == 'app_install'").groupby('month')['event'].count()
#installs по месяцам

month
1    80297
2    38078
3    36222
Name: event, dtype: int64

## Когортный анализ CR

In [42]:
df['dateins'] = df.query("event == 'app_install'").date #формирую новую колонку с датой установки по пользователю

In [196]:
df['date_purch'] = df.query("event == 'purchase'").date #формирую новую колонку с датой первой оплаты пользователя

In [208]:
kdf = df.groupby('device_id', as_index = False).agg({'dateins':'max', 'date_purch':'min'})

In [209]:
kdf = kdf.dropna()

In [210]:
kdf

,device_id,dateins,date_purch
4,4046,2020-01-04,2020-01-04
10,4120,2020-01-07,2020-01-07
11,4125,2020-01-07,2020-01-07
14,4202,2020-01-08,2020-02-27
19,4234,2020-02-19,2020-02-19
...,...,...,...
190851,35315632,2020-03-31,2020-03-31
190855,35321120,2020-03-25,2020-03-25
190866,35342310,2020-03-27,2020-03-27
190875,35367797,2020-03-29,2020-03-29


In [211]:
kdf['dif'] = kdf.date_purch - kdf.dateins

In [212]:
fft = pd.Timedelta(7, 'd')
fft

Timedelta('7 days 00:00:00')

In [213]:
kdf = kdf.query('dif <= @fft')

In [226]:
kdf

,device_id,dateins,date_purch,dif
4,4046,2020-01-04,2020-01-04,0 days
10,4120,2020-01-07,2020-01-07,0 days
11,4125,2020-01-07,2020-01-07,0 days
19,4234,2020-02-19,2020-02-19,0 days
20,4244,2020-03-19,2020-03-19,0 days
...,...,...,...,...
190851,35315632,2020-03-31,2020-03-31,0 days
190855,35321120,2020-03-25,2020-03-25,0 days
190866,35342310,2020-03-27,2020-03-27,0 days
190875,35367797,2020-03-29,2020-03-29,0 days


In [236]:
kdf.groupby('dateins')['device_id'].count().head(1)
#Когорта с наибольшими установками - 2020-01-01

dateins
2020-01-01    1408
Name: device_id, dtype: int64

In [237]:
df.groupby('dateins')['device_id'].count().head(1)
#Всего установок

dateins
2020-01-01    3579
Name: device_id, dtype: int64

In [235]:
1408/3579*100
#CR за 01.01.2020

39.340597932383346

## Каналы привлечения и Retention

In [240]:
df.groupby('utm_source')['device_id'].nunique().sort_values()
#каналы привлечения юзеров
#Больше всего пришло пользователей из яндекса

utm_source
referal          15926
facebook_ads     25959
instagram_ads    31048
vk_ads           34086
google_ads       38096
yandex-direct    40712
-                52273
Name: device_id, dtype: int64

In [267]:
df= df.drop(columns = ['regdate'])

In [275]:
rdf = df.query("event == 'register'").groupby('device_id', as_index = False).agg({'date':'min'}).rename(columns = {'date':'regdate'})

In [278]:
ndf = pd.merge(df, rdf, how = 'inner', on = 'device_id')

In [282]:
ndf.query('regdate < date').groupby('event')['device_id'].nunique().sort_values(ascending=False)
#Воронка магазина.
#Больше всего пользователей отваливается на этапе перехода в корзину

event
app_start      40991
search         40482
choose_item    37926
tap_basket     34517
purchase       24880
Name: device_id, dtype: int64

In [308]:
df.query("event == 'purchase'").groupby('utm_source')['device_id'].nunique().sort_values() / df.query("event == 'app_start'").groupby('utm_source')['device_id'].nunique().sort_values()
#CR из запуска приложения в покупку по каналам привлечения
#Самый низкий CR у Яндекса

utm_source
-                0.317556
facebook_ads     0.347382
google_ads       0.297674
instagram_ads    0.346658
referal          0.399523
vk_ads           0.362804
yandex-direct    0.295477
Name: device_id, dtype: float64

In [386]:
mdf = df.query("event == 'purchase'").groupby('device_id', as_index = False).agg({'date':'min'}) 
#Дата первой покупки пользователя

In [405]:
mdf.head()

,device_id,date
0,4014,2020-02-01
1,4046,2020-01-04
2,4060,2020-03-07
3,4120,2020-01-07
4,4125,2020-01-07


In [400]:
xdf = pd.merge(mdf, df[['device_id', 'purchase_sum', 'utm_source', 'date']], how = 'left', on = ['device_id','date'])

In [401]:
xdf = xdf.dropna()

In [404]:
xdf.head()

,device_id,date,purchase_sum,utm_source
2,4014,2020-02-01,86.0,referal
8,4046,2020-01-04,429.5,-
15,4060,2020-03-07,172.5,yandex-direct
21,4120,2020-01-07,598.5,instagram_ads
28,4125,2020-01-07,566.0,yandex-direct


In [403]:
xdf.groupby('utm_source')['purchase_sum'].median()
#расчет медианного первого чека пользователя привлеченного по разным каналам

utm_source
-                398.5
facebook_ads     389.0
google_ads       390.5
instagram_ads    393.5
referal          395.5
vk_ads           393.0
yandex-direct    392.5
Name: purchase_sum, dtype: float64

In [418]:
df.groupby('utm_source')['purchase_sum'].sum()
#Суммарная прибыль от каналов привлечения

utm_source
-                21449749.5
facebook_ads     12249901.0
google_ads       12868276.0
instagram_ads    14546969.0
referal           8837044.5
vk_ads           16389652.5
yandex-direct    13915368.0
Name: purchase_sum, dtype: float64